In [1]:
!pip install openai==0.28

In [9]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 25.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.8 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.49
    Uninstalling langchain-core-0.3.49:
      Successfully uninstalled langchain-core-0.3.49
  Attempting uninstall: langchain-text-splitters
    Found existing installation: langchain-text-splitters 0.3.7
    Uninstalling langchain-text-splitters-0.3.7:
      Successfully uninstalled langchain-text-splitters-0.3.7
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.22
    Uninstalling langchain-0.3.22:
      Successfully uninstalled langchain-0.3.22


In [2]:
# Step 2: Import libraries
import openai
import json
import os
from datetime import datetime

In [34]:
# Step 3: Set your OpenAI API key
openai.api_key = ""  # Replace with your actual key securely

# Optional: store in env
os.environ["OPENAI_API_KEY"] = openai.api_key

In [4]:
# Step 4: Convert dataset from prompt-completion to chat format

input_path = "/content/drive/MyDrive/dataset-competition/footballers_finetuning.jsonl"            # Your original file
formatted_path = "/content/drive/MyDrive/dataset-competition/footballers_finetuning_formatted_dataset.jsonl"  # Output file

with open(input_path, "r") as infile, open(formatted_path, "w") as outfile:
    for line in infile:
        data = json.loads(line)
        chat_data = {
            "messages": [
                {"role": "user", "content": data["prompt"]},
                {"role": "assistant", "content": data["completion"]}
            ]
        }
        json.dump(chat_data, outfile)
        outfile.write("\n")

print("✅ Dataset formatted for chat model.")


✅ Dataset formatted for chat model.


In [5]:
import requests

headers = {
    "Authorization": f"Bearer {openai.api_key}"
}

# Open the file in binary mode
with open(formatted_path, "rb") as f:
    response = requests.post(
        "https://api.openai.com/v1/files",
        headers=headers,
        files={"file": f},
        data={"purpose": "fine-tune"}
    )

# Convert the response to JSON and extract the file id
upload_response = response.json()
file_id = upload_response.get("id")
print("📤 Uploaded File ID:", file_id)

📤 Uploaded File ID: file-XEWG7mq6HAVZgkdScfZSFK


In [17]:
# Initiate fine-tuning job using the uploaded file ID
fine_tune_response = openai.FineTuningJob.create(
    training_file=file_id,  # Use the uploaded file ID
    model="gpt-3.5-turbo-1106",     # Use this or latest supported
    suffix="footballers-finetuning" # Provide a suffix for your fine-tuned model
)

job_id = fine_tune_response["id"]
print("🚀 Fine-tuning job started with Job ID:", job_id)

🚀 Fine-tuning job started with Job ID: ftjob-EIzpvQcjPMZUIAqs3G4ycb8a


In [31]:
# Step 7: Monitor fine-tuning job status

def check_job_status(job_id):
    job_info = openai.FineTuningJob.retrieve(job_id)
    status = job_info["status"]
    print(f"🕒 Status: {status}")
    if status == "succeeded":
        print("✅ Fine-tuning completed successfully!")
        print("Model name:", job_info["fine_tuned_model"])
    elif status == "failed":
        print("❌ Fine-tuning failed.")
    return job_info

# Re-run this cell periodically to check status
job_info = check_job_status(job_id)

🕒 Status: succeeded
✅ Fine-tuning completed successfully!
Model name: ft:gpt-3.5-turbo-1106:personal:footballers-finetuning:BIxFJYux


In [32]:
print(job_info["status"])
print(job_info["fine_tuned_model"])

succeeded
ft:gpt-3.5-turbo-1106:personal:footballers-finetuning:BIxFJYux


In [38]:
from langchain.chat_models import ChatOpenAI

llm_model = ChatOpenAI(
    model=job_info["fine_tuned_model"],  # or paste the model name manually
    openai_api_key=openai.api_key
)

response = llm_model.invoke("Create a 1-week training program for Faisal Khan")
print(response.content)

Day 1: 
- Warm-up: 10 min jog
- Strength training: 3 sets of squats, lunges, and push-ups
- Cardio: 20 min cycling

Day 2:
- Yoga: 30 min session focusing on flexibility and balance

Day 3:
- Warm-up: 10 min jump rope
- HIIT: 20 min session of high-intensity interval training
- Core: 3 sets of planks, Russian twists, and leg raises

Day 4:
- Rest day

Day 5:
- Warm-up: 10 min brisk walk
- Strength training: 3 sets of deadlifts, shoulder presses, and pull-ups
- Cardio: 20 min rowing

Day 6:
- Yoga: 30 min session focusing on strength and relaxation

Day 7:
- Active recovery: 30 min walk or light jog
